In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        k = 1
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import pandas as pd

In [ ]:
data = pd.read_csv('/kaggle/input/hand-written-digits-for-object-detection/train_label.csv')

image = data.iloc[:, 0].values
label = data.iloc[:, 1:].values

In [ ]:
def collector(images_file, label):
    image = tf.io.read_file('/kaggle/input/hand-written-digits-for-object-detection/train/train/'+images_file)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)

    labels = {'label': label[0], 'coordinates': label[1:]}
    return image, labels

In [ ]:
train = tf.data.Dataset.from_tensor_slices((image, label))

train = (
    train.shuffle(buffer_size=label.shape[0])
    .map(collector)
    .batch(batch_size=220))

In [ ]:
save_callback = tf.keras.callbacks.ModelCheckpoint(
    'checkpoint/',
    save_weights_only=True,
    monitor='accuracy',
    save_best_only=True
)

In [ ]:
rg = tf.keras.regularizers.l1(0.001)
input = tf.keras.Input(shape=(75, 75, 1))

x = tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu', kernel_regularizer=rg)(input)
x = tf.keras.layers.MaxPooling2D((3,3), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu', kernel_regularizer=rg)(x)
x = tf.keras.layers.MaxPooling2D((3,3), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(128, kernel_size=3, activation='relu', kernel_regularizer=rg)(x)
x = tf.keras.layers.MaxPooling2D((3,3), strides=(1,1), padding='valid')(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Flatten()(x)

In [ ]:
output1 = tf.keras.layers.Dense(10, activation='softmax', name="label")(x)
output2 = tf.keras.layers.Dense(4, name="coordinates")(x)

model = tf.keras.Model(inputs=input, outputs=[output1, output2])

In [ ]:
model.compile(loss={"label": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),"coordinates": 'mean_squared_error'},
              optimizer='adam', metrics=['accuracy'])

model.fit(train, epochs=15, batch_size=220, verbose=1, callbacks=[save_callback])